# Лабораторная работа «Нейронная сеть для распознавания рукописных цифр»

Цель: написать компьютерную программу на языке Python, создающую и обучающую
нейронную сеть для распознавания рукописных цифр.

Исходные данные:

1. Количество слоев нейронной сети: 3

2. Входные данные для нейронной сети: изображения размером 28 × 28 пикселов.


In [42]:
pip install wget # Пакет wget для скачивания файлов из Интернета

In [43]:
import wget
for index in range(10):
  as_str = str(index)
  as_str += ".bmp"
  wget.download('http://xyla.istu.webappz.ru/neuron/'+as_str)
wget.download(('http://xyla.istu.webappz.ru/neuron/mnist.pkl.gz'))
# база данных MNIST содержит 70 000 изображений рукописных цифр, разделенных на три набора:
# training_data – набор из 50 000 изображений предназначен для обучения нейронных сетей;
# validation_data – набор из 10 000 изображений предназначен для текущей оценки работы алгоритма обучения и подбора параметров обучения (используется в последующих лабораторных работах);
# test_data – набор из 10 000 изображений предназначен для проверки работы нейронной сетей.
# Каждый набор состоит из двух списков: списка изображений (в градациях серого) и соответствующего списка цифр в диапазоне от 0 до 9 Изображение представлено в виде одномерного numpy-массива размера 784 = 28 × 28 значений от 0 до 1, где 0 соответствует черному цвету пиксела, а 1 – белому.

'mnist.pkl.gz'

In [44]:
# В качестве функции активации для нейронов сети используется сигмоидальная функция, вычисляющая выходной сигнал искусственного нейрона
def sigmoid(z):                         # определение сигмоидальной функции активации
    return 1.0/(1.0+np.exp(-z))

In [45]:
# Данный метод осуществляет подсчет выходных сигналов нейронной сети при заданных входных сигналах.
# Параметр a является массивом n × 1, где n – количество нейронов входного слоя. Функция np.dot вычисляет произведение матриц.
# Для подсчета выходных значений нейронной сети, необходимо один раз вызвать метод feedforward, в результате чего выходные сигналы будут последовательно вычислены для всех слоев нейронной сети.
def feedforward(self, a):
    for b, w in zip(self.biases, self.weights):
        a = sigmoid(np.dot(w, a)+b)
    return a

In [46]:
# Для реализации механизма обучения создаваемой нейронной сети добавим метод SGD, который реализует стохастический градиентный спуск. Метод имеет следующие параметры:
# «Training_data» – обучающая выборка, состоящая из пар вида ( , ), где – вектор входных сигналов, а – ожидаемый вектор выходных сигналов;
# «epochs» – количество эпох обучения;
# «mini_batch_size» - размер подвыборки;
# «eta» - скорость обучения;
# «test_data» - (необязательный параметр); если данный аргумент не пуст, то программа после каждой эпохи обучения осуществляет оценку работы сети и показывает достигнутый прогресс.
def SGD(                              # Стохастический градиентный спуск
          self                        # указатель на объект класса
        , training_data               # обучающая выборка
        , epochs                      # количество эпох обучения
        , mini_batch_size             # размер подвыборки
        , eta                         # скорость обучения
        , test_data                   # тестирующая выборка
        ):
    test_data = list(test_data)                                         # создаем список объектов тестирующей выборки
    n_test = len(test_data)                                             # вычисляем длину тестирующей выборки
    training_data = list(training_data)                                 # создаем список объектов обучающей выборки
    n = len(training_data)                                              # вычисляем размер обучающей выборки
    for j in range(epochs):                                             # цикл по эпохам
        random.shuffle(training_data)                                   # перемешиваем элементы обучающей выборки
        mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]    # создаем подвыборки
        for mini_batch in mini_batches:                                                              # цикл по подвыборкам
            self.update_mini_batch(mini_batch, eta)                                                  # один шаг градиентного спуска
        print ("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test))                   # смотрим прогресс в обучении
        with open('biases.pkl', 'wb') as f:
            pickle.dump(self.biases, f)
        with open('weights.pkl', 'wb') as f:
            pickle.dump(self.weights, f)

In [47]:
# Метод update_mini_batch вычисляет градиенты для каждого прецедента в подвыборке, а затем соответствующим образом обновляет веса и смещения нейронной сети
def update_mini_batch(                    # Шаг градиентного спуска
                      self                # указатель на объект класса
                    , mini_batch          # подвыборка
                    , eta                 # скорость обучения
                    ):
    nabla_b = [np.zeros(b.shape) for b in self.biases]                  # список градиентов dC/db для каждого слоя (первоначально заполняются нулями)
    nabla_w = [np.zeros(w.shape) for w in self.weights]                 # список градиентов dC/dw для каждого слоя (первоначально заполняются нулями)
    for x, y in mini_batch:
        delta_nabla_b, delta_nabla_w = self.backprop(x, y)              # послойно вычисляем градиенты dC/db и dC/dw для текущего прецедента (x, y)
        nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]   # суммируем градиенты dC/db для различных прецедентов текущей подвыборки
        nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]   # суммируем градиенты dC/dw для различных прецедентов текущей подвыборки
    self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]   # обновляем все веса w нейронной сети
    self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]     # обновляем все смещения b нейронной сети

In [48]:
def backprop(                           # Алгоритм обратного распространения
              self                      # указатель на объект класса
            , x                         # вектор входных сигналов
            , y                         # ожидаемый вектор выходных сигналов
            ):
    nabla_b = [np.zeros(b.shape) for b in self.biases]                  # список градиентов dC/db для каждого слоя (первоначально заполняются нулями)
    nabla_w = [np.zeros(w.shape) for w in self.weights]                 # список градиентов dC/dw для каждого слоя (первоначально заполняются нулями)

    # определение переменных
    activation = x                      # выходные сигналы слоя (первоначально соответствует выходным сигналам 1-го слоя или входным сигналам сети)
    activations = [x]                   # список выходных сигналов по всем слоям (первоначально содержит только выходные сигналы 1-го слоя)
    zs = []                             # список активационных потенциалов по всем слоям (первоначально пуст)

    # прямое распространение
    for b, w in zip(self.biases, self.weights):
        z = np.dot(w, activation)+b     # считаем активационные потенциалы текущего слоя
        zs.append(z)                    # добавляем элемент (активационные потенциалы слоя) в конец списка
        activation = sigmoid(z)         # считаем выходные сигналы текущего слоя, применяя сигмоидальную функцию активации к активационным потенциалам слоя
        activations.append(activation)  # добавляем элемент (выходные сигналы слоя) в конец списка

    # обратное распространение
    delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1]) # считаем меру влияния нейронов выходного слоя L на величину ошибки (BP1)
    nabla_b[-1] = delta                 # градиент dC/db для слоя L (BP3)
    nabla_w[-1] = np.dot(delta, activations[-2].transpose()) # градиент dC/dw для слоя L (BP4)
    for l in range(2, self.num_layers):
        z = zs[-l]                      # активационные потенциалы l-го слоя (двигаемся "Положение EcoRobotics.docx"списку справа налево)
        sp = sigmoid_prime(z)           # считаем сигмоидальную функцию от активационных потенциалов l-го слоя
        delta = np.dot(self.weights[-l+1].transpose(), delta) * sp      # считаем меру влияния нейронов l-го слоя на величину ошибки (BP2)
        nabla_b[-l] = delta                                             # градиент dC/db для l-го слоя (BP3)
        nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())      # градиент dC/dw для l-го слоя (BP4)
    return (nabla_b, nabla_w)

In [49]:
# Метод возвращает количество прецедентов тестирующей выборки, для которых нейронная сеть выдает правильный результат
# Тестирующая выборка состоит из пар (x, y), где x – вектор размерности 784, содержащий изображение цифры, а y – целое числовое значение цифры, изображенной на картинке
# Ответ нейронной сети определяется как номер нейрона в выходном слое, имеющего наибольшее значение функции активации
# Метод evaluate вызывается в методе SGD после завершения очередной эпохи обучения
def evaluate(self, test_data): # Оценка прогресса в обучении
    test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
    return sum(int(x == y) for (x, y) in test_results)

In [50]:
# Вычисление частных производных стоимостной функции по выходным сигналам последнего слоя
def cost_derivative(self, output_activations, y): # Вычисление частных производных стоимостной функции по выходным сигналам последнего слоя
    return (output_activations-y)

In [51]:
def sigmoid_prime(z):  # Производная сигмоидальной функции
    return sigmoid(z)*(1-sigmoid(z))

In [52]:
# Функции для работы с базой данных MNIST
import gzip # библиотека для сжатия и распаковки файлов gzip и gunzip.
import pickle # библиотека для сохранения и загрузки сложных объектов Python.
import numpy as np # библиотека для работы с матрицами

def load_data():
    f = gzip.open('mnist.pkl.gz', 'rb') # открываем сжатый файл gzip в двоичном режиме
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1') # загружам таблицы из файла
    f.close() # закрываем файл
    return (training_data, validation_data, test_data)

In [53]:
# Для преобразования числа в вектор-столбец (10-мерный numpy массив), используется следующая функция vectorized_result.
def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1
    return e

In [54]:
# Для использования базы данных MNIST в нашей программе необходимо скорректировать форматы наборы training_data, validation_data и test_data
# Это делается в функции load_data_wrapper
def load_data_wrapper():
    tr_d, va_d, te_d = load_data()                                      # инициализация наборов данных в формате MNIST
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]        # преобразование массивов размера 1 на 784 к массивам размера 784 на 1
    training_results = [vectorized_result(y) for y in tr_d[1]]          # представление цифр от 0 до 9 в виде массивов размера 10 на 1
    training_data = zip(training_inputs, training_results)              # формируем набор обучающих данных из пар (x, y)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]      # преобразование массивов размера 1 на 784 к массивам размера 784 на 1
    validation_data = zip(validation_inputs, va_d[1])                   # формируем набор данных проверки из пар (x, y)
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]            # преобразование массивов размера 1 на 784 к массивам размера 784 на 1
    test_data = zip(test_inputs, te_d[1])                               # формируем набор тестовых данных из пар (x, y)
    return (training_data, validation_data, test_data)

training_data, validation_data,test_data = load_data_wrapper()
# Данная функция преобразует training_data в список, содержащий 50 000 пар (x, y), где x является 784-мерным numpy-массивом, содержащим входное изображение
# а y – это 10-мерный numpy-массив, представляющий собой вектор, у которого координата с порядковым номером, соответствующим цифре на изображении
# равняется единице, а остальные координаты нулевые. Аналогичные преобразования делаются для наборов validation_data и test_data.

In [55]:
import numpy as np # библиотека функций для работы с матрицами
import pickle

def sigmoid(z): # определение сигмоидальной функции активации
    return 1.0/(1.0+np.exp(-z))

In [56]:
from IPython.lib.display import exists
"""""""""""""""""""""""""""""""""""""""
Модуль создания и обучения нейронной сети для распознавания рукописных цифр с использованием метода градиентного спуска.
Группа: АСУб-20-2
ФИО:Арбакова Анастасия Вячеславовна
"""""""""""""""""""""""""""""""""""""""
#### Библиотеки
# Стандартные библиотеки
import random           # библиотека функций для генерации случайных значений
import os.path

# Сторонние библиотеки
import numpy as np      # библиотека функций для работы с матрицами

""" ---Раздел описаний--- """
""" --Описание класса Network--"""
class Network(object):                        # используется для описания нейронной сети
    def __init__(self, sizes):                # конструктор класса, self – указатель на объект класса, sizes – список размеров слоев нейронной сети
        self.num_layers = len(sizes)          # задаем количество слоев нейронной сети
        self.sizes = sizes                    # задаем список размеров слоев нейронной сети
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]                         # задаем случайные начальные смещения
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]    # задаем случайные начальные веса связей
        if(os.path.isfile('biases.pkl')):
          with open('biases.pkl', 'rb') as f: #открытие файла со смещениями
            self.biases = pickle.load(f)
        if(os.path.isfile('weights.pkl')):
          with open('weights.pkl', 'rb') as f:#открытие файла с весами
            self.weights = pickle.load(f)
    def sigmoid(z):                            # определение сигмоидальной функции активации
        return 1.0/(1.0+np.exp(-z))
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def evaluate(self, test_data):             # Оценка прогресса в обучении
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y): # Вычисление частных производных стоимостной функции по выходным сигналам последнего слоя
        return (output_activations-y)

    def backprop(                     # Алгоритм обратного распространения
              self                    # указатель на объект класса
            , x                       # вектор входных сигналов
            , y                       # ожидаемый вектор выходных сигналов
            ):
        nabla_b = [np.zeros(b.shape) for b in self.biases]      # список градиентов dC/db для каждого слоя (первоначально заполняются нулями)
        nabla_w = [np.zeros(w.shape) for w in self.weights]     # список градиентов dC/dw для каждого слоя (первоначально заполняются нулями)

        # определение переменных
        activation = x                # выходные сигналы слоя (первоначально соответствует выходным сигналам 1-го слоя или входным сигналам сети)
        activations = [x]             # список выходных сигналов по всем слоям (первоначально содержит только выходные сигналы 1-го слоя)
        zs = []                       # список активационных потенциалов по всем слоям (первоначально пуст)

        # прямое распространение
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b                   # считаем активационные потенциалы текущего слоя
            zs.append(z)                                  # добавляем элемент (активационные потенциалы слоя) в конец списка
            activation = sigmoid(z)                       # считаем выходные сигналы текущего слоя, применяя сигмоидальную функцию активации к активационным потенциалам слоя
            activations.append(activation)                # добавляем элемент (выходные сигналы слоя) в конец списка

        # обратное распространение
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])    # считаем меру влияния нейронов выходного слоя L на величину ошибки (BP1)
        nabla_b[-1] = delta                                                         # градиент dC/db для слоя L (BP3)
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())                    # градиент dC/dw для слоя L (BP4)
        for l in range(2, self.num_layers):
            z = zs[-l]                                                              # активационные потенциалы l-го слоя (двигаемся "Положение EcoRobotics.docx"списку справа налево)
            sp = sigmoid_prime(z)                                                   # считаем сигмоидальную функцию от активационных потенциалов l-го слоя
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp              # считаем меру влияния нейронов l-го слоя на величину ошибки (BP2)
            nabla_b[-l] = delta                                                     # градиент dC/db для l-го слоя (BP3)
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())              # градиент dC/dw для l-го слоя (BP4)
        return (nabla_b, nabla_w)

    def update_mini_batch(                    # Шаг градиентного спуска
                      self                    # указатель на объект класса
                    , mini_batch              # подвыборка
                    , eta                     # скорость обучения
                    ):
        nabla_b = [np.zeros(b.shape) for b in self.biases]                          # список градиентов dC/db для каждого слоя (первоначально заполняются нулями)
        nabla_w = [np.zeros(w.shape) for w in self.weights]                         # список градиентов dC/dw для каждого слоя (первоначально заполняются нулями)
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)                      # послойно вычисляем градиенты dC/db и dC/dw для текущего прецедента (x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]           # суммируем градиенты dC/db для различных прецедентов текущей подвыборки
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]           # суммируем градиенты dC/dw для различных прецедентов текущей подвыборки
        self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)] # обновляем все веса w нейронной сети
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)] # обновляем все смещения b нейронной сети

    def SGD(                                  # Стохастический градиентный спуск
          self                                # указатель на объект класса
        , training_data                       # обучающая выборка
        , epochs                              # количество эпох обучения
        , mini_batch_size                     # размер подвыборки
        , eta                                 # скорость обучения
        , test_data                           # тестирующая выборка
        ):
        test_data = list(test_data)           # создаем список объектов тестирующей выборки
        n_test = len(test_data)               # вычисляем длину тестирующей выборки
        training_data = list(training_data)   # создаем список объектов обучающей выборки
        n = len(training_data)                # вычисляем размер обучающей выборки
        for j in range(epochs):               # цикл по эпохам
            random.shuffle(training_data)     # перемешиваем элементы обучающей выборки
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)] # создаем подвыборки
            for mini_batch in mini_batches:   # цикл по подвыборкам
                self.update_mini_batch(mini_batch, eta) # один шаг градиентного спуска
            print ("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test)) # смотрим прогресс в обучении
            with open('biases.pkl', 'wb') as f:
                pickle.dump(self.biases, f)
            with open('weights.pkl', 'wb') as f:
                pickle.dump(self.weights, f)
        """ --Конец описания класса Network--"""
""" --- Конец раздела описаний--- """

""" ---Тело программы--- """
net = Network([2, 3, 1])                     # создаем нейронную сеть из трех слоев
""" ---Конец тела программы--- """
""" Вывод результата на экран: """
print('Сеть net:')
print('Количетво слоев:', net.num_layers)
for i in range(net.num_layers):
    print('Количество нейронов в слое', i,':',net.sizes[i])
for i in range(net.num_layers-1):
    print('W_',i+1,':')
    print(np.round(net.weights[i],2))
    print('b_',i+1,':')
    print(np.round(net.biases[i],2))

Сеть net:
Количетво слоев: 3
Количество нейронов в слое 0 : 2
Количество нейронов в слое 1 : 3
Количество нейронов в слое 2 : 1
W_ 1 :
[[ 0.07  0.95 -0.86 ... -0.9  -2.81 -1.61]
 [-0.46  0.52  0.26 ... -0.6  -0.78  1.15]
 [ 0.02  1.24  0.09 ... -0.19 -0.16  0.86]
 ...
 [ 1.2   0.11  1.32 ...  0.41 -0.67 -0.33]
 [-0.05  0.74 -0.3  ...  1.13  0.27 -2.06]
 [ 0.5   0.42  0.47 ...  1.54  0.63 -1.48]]
b_ 1 :
[[ 5.11]
 [-5.53]
 [-0.12]
 [-0.4 ]
 [-0.39]
 [-0.61]
 [-0.24]
 [ 2.34]
 [ 3.17]
 [-2.37]
 [-1.4 ]
 [ 1.83]
 [ 4.69]
 [ 0.83]
 [ 0.99]
 [-5.41]
 [ 3.03]
 [-0.58]
 [-4.87]
 [ 3.88]
 [ 2.6 ]
 [-1.43]
 [ 0.54]
 [ 0.54]
 [-4.12]
 [-3.72]
 [-3.17]
 [-0.17]
 [ 1.04]
 [-0.05]]
W_ 2 :
[[ 7.500e-01 -2.900e+00 -3.600e+00  1.480e+00 -4.260e+00  2.600e+00
  -1.560e+00 -2.760e+00  1.000e+00  6.400e-01  1.670e+00  2.470e+00
  -2.130e+00  2.580e+00 -6.550e+00 -4.620e+00  3.230e+00  5.400e+00
   3.940e+00 -3.080e+00 -3.430e+00 -4.570e+00  4.900e-01  1.350e+00
  -4.710e+00 -9.400e-01 -7.500e-01  3.760e+0

In [57]:
net = Network([784, 30, 10])
# Параметры, указанные при вызове данного метода, определяют топологию создаваемой сети
# Таким образом, в результате выполнения команды будет создана сеть, состоящая из трех слоев:
# входной слой сети состоит из 784-х нейронов
# внутренний слой из 30 нейронов
# выходной слой из 10 нейронов

In [58]:
net.SGD(training_data, 20, 10, 3.0, test_data=test_data) # процедура обучения созданной нейронной сети, включающая 30 эпох
# Параметры, указанные при вызове метода SGD:
# обучающая выборка
# количество эпох обучения
# размер подвыборки
# скорость обучения
# тестирующая выборка
# Обучение может занять несколько минут. В ходе обучения будет выдаваться информация о пройденных эпохах.
# Для каждой эпохи выводится отношение количества правильно распознанных цифр к общему количеству цифр в тестовой выборке.
# Например, запись Epoch 6: 9374 / 10000 говорит о том, что в результате
# эпохи обучения с номером 6 достигнута точность распознавания ≈ 0.94, что составляет 94%.

Epoch 0: 9518 / 10000
Epoch 1: 9500 / 10000
Epoch 2: 9521 / 10000
Epoch 3: 9502 / 10000
Epoch 4: 9478 / 10000
Epoch 5: 9523 / 10000
Epoch 6: 9516 / 10000
Epoch 7: 9515 / 10000
Epoch 8: 9521 / 10000
Epoch 9: 9492 / 10000
Epoch 10: 9505 / 10000
Epoch 11: 9518 / 10000
Epoch 12: 9509 / 10000
Epoch 13: 9522 / 10000
Epoch 14: 9522 / 10000
Epoch 15: 9510 / 10000
Epoch 16: 9505 / 10000
Epoch 17: 9509 / 10000
Epoch 18: 9531 / 10000
Epoch 19: 9515 / 10000


In [59]:
from PIL import Image
import numpy

for index in range(10):
    as_str = str(index)
    as_str += ".bmp"
    im=Image.open(as_str)
    testArray=numpy.ndarray(shape=(784, 1), dtype=float, order='F')       #создание контейнера для копирования значения пикселей изображения

    #операция копирования в контейнер пикселей изображения
    for i2 in range(28):
        for i1 in range(28):
            if (im.getpixel((i1, i2))) / 255 > 0:
                #вынужден был применять костыль, т.к. 1 в значении пикселей нейросеть неадекватно воспринимает
                testArray[i1 + i2 * 28, 0] = (im.getpixel((i1, i2))) / 255 - 0.001
            else:
                testArray[i1 + i2 * 28,0] = (im.getpixel((i1, i2))) / 255

    netSm = Network([784, 30, 10])                #создаем нейросеть с обученными весами и смещениями
    print(as_str)
    display(im)
    print(netSm.feedforward(testArray))           #распознаем изображение с помощью нейросети
    print()

0.bmp


[[9.99512219e-01]
 [2.46478501e-12]
 [8.78418864e-06]
 [5.42447769e-10]
 [2.60051566e-08]
 [1.04487752e-02]
 [9.90068022e-09]
 [1.16790606e-06]
 [1.62679131e-07]
 [8.05429722e-05]]

1.bmp


[[5.53773579e-09]
 [7.25042413e-14]
 [1.36952375e-09]
 [9.00062941e-03]
 [4.37264293e-09]
 [1.68181928e-12]
 [5.51030234e-15]
 [2.05419052e-01]
 [2.04516813e-08]
 [1.90338332e-01]]

2.bmp


[[1.12047242e-10]
 [3.81427325e-06]
 [8.91922843e-06]
 [6.71435274e-01]
 [1.60840873e-06]
 [1.18577156e-06]
 [2.07982884e-16]
 [2.06133626e-02]
 [6.25740341e-10]
 [1.69021736e-04]]

3.bmp


[[1.77103561e-06]
 [4.25526236e-15]
 [3.99119347e-01]
 [9.82824526e-01]
 [3.30505741e-10]
 [1.46205048e-06]
 [3.45720399e-14]
 [9.62474393e-10]
 [2.84840829e-06]
 [3.59503242e-03]]

4.bmp


...

[[4.01202139e-11]
 [1.39436026e-18]
 [1.41594162e-05]
 [2.09985916e-06]
 [2.01625556e-01]
 [8.81619980e-05]
 [1.44111852e-05]
 [1.94102242e-08]
 [2.77622660e-01]
 [7.60010057e-03]]

6.bmp


[[6.68808807e-06]
 [1.29616101e-11]
 [2.72456499e-03]
 [1.36483641e-09]
 [2.23389707e-07]
 [5.97141448e-01]
 [6.27974786e-01]
 [1.25903613e-15]
 [3.27020579e-08]
 [2.88874981e-07]]

7.bmp


[[5.75607896e-09]
 [2.90310514e-14]
 [7.54450759e-01]
 [2.98719463e-03]
 [1.57299200e-13]
 [1.38876564e-06]
 [8.59854333e-11]
 [6.12889272e-08]
 [1.12422289e-05]
 [5.42513001e-06]]

8.bmp


[[1.17074870e-07]
 [1.71133292e-12]
 [4.48532805e-04]
 [1.94910389e-04]
 [6.43555253e-08]
 [4.95450358e-05]
 [2.00873128e-08]
 [2.23021797e-11]
 [9.99892916e-01]
 [4.15180315e-02]]

9.bmp


[[3.03664315e-08]
 [3.83270096e-09]
 [1.99643706e-08]
 [2.91352998e-02]
 [1.63147940e-09]
 [9.11965932e-01]
 [1.73962873e-06]
 [1.04941332e-15]
 [1.26006615e-01]
 [8.78205646e-06]]

